In [3]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_excel('C:/Users/Ted/Downloads/FinancialSample.xlsx')
#df = pd.read_csv('C:/Users/Ted/Downloads/FinancialSample.csv')

target = 'Units Sold'         #Segment and Units Sold are good test samples
showCrosstab = False
showGroupMeans = False        # Not working for now due to the possibility of the order of object and number to be swapped in ANOVA
includeExplanations = True    # Not implemented

correlation_categories = {
    "small": [],
    "medium": [],
    "large": []
}

# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if target != column:
        #object vs object: chi2
        # this test compares observed and expected frequencies of the crosstab
        if df[target].dtype == 'object' and df[column].dtype == 'object':
            ct = pd.crosstab(df[target], df[column])
            if showCrosstab == True:
                print(ct)
                print()
            
            chi2, p, dof, expected = stats.chi2_contingency(ct)
            critical_value = stats.chi2.ppf(1 - 0.05, dof)
            
                    
            if p < 0.05:
                if chi2 > critical_value:
                    
                    print(f'Target: {target}')
                    print(f'Column: {column}')
                    print()
                    
                    n = np.sum(ct.values) 
                    cramers_v = np.sqrt(chi2 / (n * (min(ct.shape) - 1)))
    
                    if cramers_v > 0.5:
                        doc = 'Large'
                
                    elif cramers_v > 0.3:
                        doc = 'Medium'
                    elif cramers_v > 0.1:
                        doc = 'Small'
                    else:
                        doc = 'None'

                    print(f'chi2: {chi2}')
                    print(f'Critical Value: {critical_value}')          # this is the value that chi squared must be below or above to determine if it is high or low
                    print(f'p: {p}')
                    
                    print()    
                    print('Correlation:', doc)
                    print(f'Cramers V: {cramers_v}')            # note that Cramers V works but is not suited towards 2x2 contingency tables
            
                    #print('p value is below the significance threshold of 0.05')
                    #print('chi2 value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                    print()
                    print('---------------------------------------------------------')
                    print()
                    
                else: 
                    pass
                    #print('No Correlation')
                    #print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('chi2 value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                pass
                #print('No Correlation')
                #print()
                #print('p value is above the significance threshold of 0.05')
            
            
            
            

            
        #numerical vs numerical: pearsons or spearmans
        if (df[target].dtype == 'float64' or df[target].dtype == 'int64') and (df[column].dtype == 'float64' or df[column].dtype == 'int64'):
            
            x, p = stats.pearsonr(df[target], df[column])       # can use either pearsonr or spearmanr depending on how the data is distributed
                         
            
            if p < 0.05:
                
                if x >= 0.5:
                    doc = 'Large Positive'
                    correlation_categories["large"].append(column)
                elif x >= 0.3:
                    doc = 'Medium Positive'
                    correlation_categories["medium"].append(column)
                elif x >= 0.1:
                    doc = 'Small Positive'
                    correlation_categories["small"].append(column)
                elif x < 0.1 and x > -0.1:
                    doc = 'None'
                elif x <= -0.1 and x > -0.3:
                    doc = 'Small Negative'
                    correlation_categories["small"].append(column)
                elif x <= -0.3 and x > -0.5:
                    doc = 'Medium Negative'
                    correlation_categories["medium"].append(column)
                elif x <= -0.5 and x >= -1:
                    doc = 'Large Negative'
                    correlation_categories["large"].append(column)
                
                print(f'Target: {target}')
                print(f'Column: {column}')
                print()

                print(f'x: {x}')
                print(f'p: {p}')

                print()
                
                print('Correlation:', doc)
                #print('p value is below the significance threshold of 0.05')    
                
                print()
                print('---------------------------------------------------------')
                print()
                
            else:
                pass
                #print('No Correlation')
                #print('Reason: p value is above the significance threshold of 0.05')
            
            
            
            
            
        #object vs numerical: ANOVA
        #There can be no correlation between nominal vs numerical, only something similar 
        if (df[target].dtype == 'object' and (df[column].dtype == 'float64' or df[column].dtype == 'int64')) or ((df[target].dtype == 'float64' or df[target].dtype == 'int64') and df[column].dtype == 'object'):    
            
            if df[target].dtype == 'object':
                grouped_data = df.groupby(target)[column].apply(list)
            else:
                grouped_data = df.groupby(column)[target].apply(list)

            # Convert the grouped data into a list of arrays suitable for ANOVA
            data_arrays = [group for group in grouped_data]

            #print(*data_arrays)
            #print(grouped_data)
            #print(grouped_data)
                
            # Perform ANOVA
            f, p = stats.f_oneway(*data_arrays)
            n = len(df.index)        # number of rows
            # Finding degrees of freedom for both numerator and demoninator, then using those to help find critical value
            dfn, dfd = (len(grouped_data) - 1), (n - 2)      # (2-1) because we are only comparing 2 groups, len(df.index) is number of rows
            critical_value = stats.f.ppf(1 - 0.05, dfn, dfd)
    
            
            # Calculating the 3 values here that are needed to find the eta^2
            # Calculate Total Sum of Squares (TSS)  
            # all_data = np.concatenate(grouped_data)
            all_data = []
            for group in grouped_data:
                all_data.extend(group)
            all_data = np.array(all_data)
            grand_mean = np.mean(all_data)
            tss = np.sum((all_data - grand_mean) ** 2)

            # Calculate Within-Group Sum of Squares (WSS)
            wss = sum(sum((group - np.mean(group)) ** 2) for group in grouped_data)

            # Calculate Between-Group Sum of Squares (BSS)
            bss = tss - wss

            # Calculate Eta-squared (η²)
            eta_squared = bss / tss
            
            
            if p < 0.05:
                if f > critical_value:
                    
                    if eta_squared > 0.14:
                        doc = 'Large'
                        correlation_categories["large"].append(column)
                    elif eta_squared > 0.06:
                        doc = 'Medium'
                        correlation_categories["medium"].append(column)
                    elif eta_squared > 0.01:
                        doc = 'Small'
                        correlation_categories["small"].append(column)
                    else:
                        doc = 'None'
                        
                    print(f'Target: {target}')
                    print(f'Column: {column}')  
                    
                    if showGroupMeans == True:
                        print()
                        print('Table of Means by Each Group')
                        print()
                        group_means = df.groupby(column)[target].mean().reset_index()
                        print(round(group_means, 2))
                    
                    print()
                    print("f:", f)
                    print(f"Critical Value: {critical_value}", )
                    print("p:", p)
                    print()
                    
                    print("Eta-squared:", eta_squared)     
                    print()
                    
                    print('Correlation:', doc)
                    
                    print()
                    #print('p value is below the significance threshold of 0.05')
                    #print('f value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                    print('---------------------------------------------------------')
                    print()    
                    
                else:
                    pass
                    #print('No Correlation')
                    #print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('f value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                pass
                #print('No Correlation')
                #print()
                #print('p value is above the significance threshold of 0.05')
            
            
            

Target: Units Sold
Column: Country

f: 4.446188124281453
Critical Value: 2.384692685857469
p: 0.0014863609272251354

Eta-squared: 0.024951084369888396

Correlation: Small

---------------------------------------------------------

Target: Units Sold
Column: Gross Sales

x: 0.32722066254045745
p: 6.232960229375536e-19

Correlation: Medium Positive

---------------------------------------------------------

Target: Units Sold
Column: Discounts

x: 0.2530479774656188
p: 1.0911815841023358e-11

Correlation: Small Positive

---------------------------------------------------------

Target: Units Sold
Column: Sales

x: 0.3269139725274527
p: 6.747469128497393e-19

Correlation: Medium Positive

---------------------------------------------------------

Target: Units Sold
Column: COGS

x: 0.33169439250128174
p: 1.9397416204430017e-19

Correlation: Medium Positive

---------------------------------------------------------

Target: Units Sold
Column: Profit

x: 0.2284369068302451
p: 9.71074565606

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_excel('C:/Users/Ted/Downloads/FinancialSample.xlsx')
#df = pd.read_csv('C:/Users/Ted/Downloads/FinancialSample.csv')

target = 'Segment'
showCrosstab = False


# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if target != column:
        #object vs object: chi2
        # this test compares observed and expected frequencies of the crosstab
        if df[target].dtype == 'object' and df[column].dtype == 'object':
            print(f'Target: {target}')
            print(f'Column: {column}')
            print()
            ct = pd.crosstab(df[target], df[column])
            if showCrosstab == True:
                print(ct)
                print()
            
            chi2, p, dof, expected = stats.chi2_contingency(ct)
            critical_value = stats.chi2.ppf(1 - 0.05, dof)
            
            n = np.sum(ct.values) 
            v = np.sqrt(chi2 / (n * (min(ct.shape) - 1)))
            
            print(f'chi2: {chi2}')
            print(f'Critical Value: {critical_value}')          # this is the value that chi squared must be below or above to determine if it is high or low
            print(f'p: {p}')
            
            print()
            print('Conclusion:')
            
            if p < 0.05:
                if chi2 > critical_value:
                    print('Correlation')
                    print(f'Cramers V: {v}')            # note that Cramers V works but is not suited towards 2x2 contingency tables
            
                    print()
                    print('p value is below the significance threshold of 0.05')
                    print('chi2 value is greater than critical value')
                    print('the difference between observed and expected frequencies is too large to be attributed to chance')

                else: 
                    print('No Correlation')
                    print()
                    print('p value is above the significance threshold of 0.05')
                    print('chi2 value is less than critical value')
                    print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                print('No Correlation')
                print()
                print('p value is above the significance threshold of 0.05')
            
            print()
            print('---------------------------------------------------------')
            print()
            

Target: Segment
Column: Country

chi2: 0.0
Critical Value: 26.29622760486423
p: 1.0

Conclusion:
No Correlation

p value is above the significance threshold of 0.05

---------------------------------------------------------

Target: Segment
Column: Product

chi2: 11.645756859692945
Critical Value: 31.410432844230918
p: 0.9277293332910025

Conclusion:
No Correlation

p value is above the significance threshold of 0.05

---------------------------------------------------------

Target: Segment
Column: Discount Band

chi2: 19.200076992281698
Critical Value: 15.50731305586545
p: 0.013825488896096009

Conclusion:
Correlation
Cramers V: 0.1218103937222306

p value is below the significance threshold of 0.05
chi2 value is greater than critical value
the difference between observed and expected frequencies is too large to be attributed to chance

---------------------------------------------------------

Target: Segment
Column: Month Name

chi2: 0.0
Critical Value: 60.480886582336446
p: 1.0

C

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_excel('C:/Users/Ted/Downloads/FinancialSample.xlsx')
#df = pd.read_csv('C:/Users/Ted/Downloads/FinancialSample.csv')

target = 'Units Sold'

# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if target != column: 
        #numerical vs numerical: pearsons or spearmans
        if (df[target].dtype == 'float64' or df[target].dtype == 'int64') and (df[column].dtype == 'float64' or df[column].dtype == 'int64'):
            x, p = stats.pearsonr(df[target], df[column])       # can use either pearsonr or spearmanr depending on how the data is distributed
            print(f'Target: {target}')
            print(f'Column: {column}')
            print()
            
            print(f'x: {x}')
            print(f'p: {p}')
            
            print()
            
            if p < 0.05:
                print('Correlation')
                print('p value is below the significance threshold of 0.05')    
            else:
                print('No Correlation')
                print('Reason: p value is above the significance threshold of 0.05')
            
            print()
            print('---------------------------------------------------------')
            print()


Target: Units Sold
Column: Manufacturing Price

x: -0.029643972853024458
p: 0.4335823652264368

No Correlation
Reason: p value is above the significance threshold of 0.05

---------------------------------------------------------

Target: Units Sold
Column: Sale Price

x: -0.0650658134215006
p: 0.0853913408043615

No Correlation
Reason: p value is above the significance threshold of 0.05

---------------------------------------------------------

Target: Units Sold
Column: Gross Sales

x: 0.32722066254045745
p: 6.232960229375536e-19

Correlation
p value is below the significance threshold of 0.05

---------------------------------------------------------

Target: Units Sold
Column: Discounts

x: 0.2530479774656188
p: 1.0911815841023358e-11

Correlation
p value is below the significance threshold of 0.05

---------------------------------------------------------

Target: Units Sold
Column: Sales

x: 0.3269139725274527
p: 6.747469128497393e-19

Correlation
p value is below the significan

In [90]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_excel('C:/Users/Ted/Downloads/FinancialSample.xlsx')
#df = pd.read_csv('C:/Users/Ted/Downloads/FinancialSample.csv')

target = 'Units Sold'
showGroupMeans = False

# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if target != column: 
        #object vs numerical: ANOVA
        #There can be no correlation between nominal vs numerical, only something similar 
        if ((df[target].dtype == 'float64' or df[target].dtype == 'int64') and df[column].dtype == 'object'):    
            
            print(f'Target: {target}')
            print(f'Column: {column}')  
            
            if df[target].dtype == 'object':
                grouped_data = df.groupby(target)[column].apply(list)
            else:
                grouped_data = df.groupby(column)[target].apply(list)

            # Convert the grouped data into a list of arrays suitable for ANOVA
            data_arrays = [group for group in grouped_data]

            if showGroupMeans == True:
                print()
                print('Table of Means by Each Group')
                print()
                group_means = df.groupby(column)[target].mean().reset_index()
                print(round(group_means, 2))

            #print(*data_arrays)
            #print(grouped_data)
            #print(grouped_data)
                
            # Perform ANOVA
            f, p = stats.f_oneway(*data_arrays)
            n = len(df.index)        # number of rows
            # Finding degrees of freedom for both numerator and demoninator, then using those to help find critical value
            dfn, dfd = (len(grouped_data) - 1), (n - 2)      # (2-1) because we are only comparing 2 groups, len(df.index) is number of rows
            critical_value = stats.f.ppf(1 - 0.05, dfn, dfd)
    
            
            # Calculating the 3 values here that are needed to find the eta^2
            # Calculate Total Sum of Squares (TSS)  
            # all_data = np.concatenate(grouped_data)
            all_data = []
            for group in grouped_data:
                all_data.extend(group)
            all_data = np.array(all_data)
            grand_mean = np.mean(all_data)
            tss = np.sum((all_data - grand_mean) ** 2)

            # Calculate Within-Group Sum of Squares (WSS)
            wss = sum(sum((group - np.mean(group)) ** 2) for group in grouped_data)

            # Calculate Between-Group Sum of Squares (BSS)
            bss = tss - wss

            # Calculate Eta-squared (η²)
            eta_squared = bss / tss
            
            
            print()
            print("f:", f)
            print(f"Critical Value: {critical_value}", )
            print("p:", p)
            print()
            
            
            if eta_squared > 0.14:
                doc = 'Large'
            elif eta_squared > 0.06:
                doc = 'Medium'
            elif eta_squared > 0.01:
                doc = 'Small'
            else:
                doc = 'None'
            
            if p < 0.05:
                if f > critical_value:
                    print('Correlation:', doc)
                    print("Eta-squared:", eta_squared)     
                    print()
                    #print('p value is below the significance threshold of 0.05')
                    #print('f value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                else: 
                    print('No Correlation')
                    print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('f value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                print('No Correlation')
                print()
                print('p value is above the significance threshold of 0.05')
            
            print()
            print('---------------------------------------------------------')
            print()    

In [ ]:


df[target].dtype == 'object' and (df[column].dtype == 'float64' or df[column].dtype == 'int64'))

In [26]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_excel('C:/Users/Ted/Downloads/FinancialSample.xlsx')
#df = pd.read_csv('C:/Users/Ted/Downloads/FinancialSample.csv')

target = 'Country'

# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if target != column:       
        #anything vs datetime: Time Series Analysis
        if (df[target].dtype == 'object' or df[target].dtype == 'float64' or df[target].dtype == 'int64') and df[column].dtype == 'datetime64[ns]':
        #or df[target].dtype == 'datetime64[ns]' and (df[column].dtype == 'object' or df[column].dtype == 'float64' or df[column].dtype == 'int64')
            
            print(f'Target: {target}')
            print(f'Column: {column}')  
            print('Anything vs datetime')
            # need to add analysis, maybe graph, summary statistics, crosstab, etc?
            
            

Target: Country
Column: Date
Anything vs datetime


In [27]:
# can change this to report words back such as categorical variable, then move this to the beginning as well as
# combine it with summary statistics

print()
for column in df.columns:
    print(f'{df[column].dtype} - {column}')


object - Segment
object - Country
object - Product
object - Discount Band
float64 - Units Sold
int64 - Manufacturing Price
int64 - Sale Price
float64 - Gross Sales
float64 - Discounts
float64 - Sales
float64 - COGS
float64 - Profit
datetime64[ns] - Date
int64 - Month Number
object - Month Name
int64 - Year


In [28]:
#ct = pd.crosstab(df['Country'], df['Product'])
#print(ct)
#print()

#stat, p, dof, expected = stats.chi2_contingency(ct)
#critical_value = stats.chi2.ppf(1 - 0.05, dof)

#print(stat)
#print(critical_value)          # this is the value that chi squared must be below or above to determine if it is high or low
#print(p)


#tvar = df[target]           # target variable


#res = stats.spearmanr(x, y)
#print(res)



#segment_data = df['Segment']
#sales_data = df['Sales']
#gross_sales_data = df['Gross Sales']


#f_value, p_value = stats.f_oneway(segment_data, sales_data, gross_sales_data)
#f_value, p_value = stats.f_oneway(df['Segment'], df['Sales'], df['Gross Sales'])
#print("F-value:", f_value, "P-value:", p_value)


#import numpy as np
#import scipy.stats

#x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
#y = np.array([2, 1, 4, 5, 8, 12, 18, 25, 96, 48])

#scipy.stats.pearsonr(x, y)    # Pearson's r
#scipy.stats.spearmanr(x, y)   # Spearman's rho
#scipy.stats.kendalltau(x, y)  # Kendall's tau


#out = scipy.stats.spearmanr(x, y)
#print(out)






In [29]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Sample DataFrame with categorical features
data = {'X1': [1, 2, 3, 4, 5],
        'X2': [5, 4, 3, 2, 1],
        'X3': [2, 3, 4, 5, 6],
        'y': [1, 0, 1, 0, 1]}
df = pd.DataFrame(data)

# Separate features and target
X = df.drop('y', axis=1)
y = df['y']

# Apply SelectKBest with chi-squared test
k = 2  # Number of top features to select
chi2_selector = SelectKBest(chi2, k=k)
X_kbest = chi2_selector.fit_transform(X, y)

# Get selected feature names
selected_features = [X.columns[i] for i in chi2_selector.get_support(indices=True)]

# Results
print("Selected features based on chi-squared test:", selected_features)

Selected features based on chi-squared test: ['X2', 'X3']
